In [6]:
import pandas as pd

# Define the file paths using raw strings
train_file_path = r'D:\manga\MACHINE,DEEP LEARNING\binary classification project\train.csv'
test_file_path = r'D:\manga\MACHINE,DEEP LEARNING\binary classification project\test.csv'

# Load the CSV files into pandas DataFrames
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

# Display the first 5 rows of each DataFrame to confirm they loaded correctly
print("Train DataFrame head:")
print(train_df.head())

print("\nTest DataFrame head:")
print(test_df.head())

Train DataFrame head:
   id  age          job  marital  education default  balance housing loan  \
0   0   42   technician  married  secondary      no        7      no   no   
1   1   38  blue-collar  married  secondary      no      514      no   no   
2   2   36  blue-collar  married  secondary      no      602     yes   no   
3   3   27      student   single  secondary      no       34     yes   no   
4   4   26   technician  married  secondary      no      889     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0  cellular   25   aug       117         3     -1         0  unknown  0  
1   unknown   18   jun       185         1     -1         0  unknown  0  
2   unknown   14   may       111         2     -1         0  unknown  0  
3   unknown   28   may        10         2     -1         0  unknown  0  
4  cellular    3   feb       902         1     -1         0  unknown  1  

Test DataFrame head:
       id  age            job  marital  education

In [7]:
# Check the training data
print("Training Data Info:")
train_df.info()

print("\n" + "="*50 + "\n")

# Check the test data
print("Test Data Info:")
test_df.info()

Training Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         750000 non-null  int64 
 1   age        750000 non-null  int64 
 2   job        750000 non-null  object
 3   marital    750000 non-null  object
 4   education  750000 non-null  object
 5   default    750000 non-null  object
 6   balance    750000 non-null  int64 
 7   housing    750000 non-null  object
 8   loan       750000 non-null  object
 9   contact    750000 non-null  object
 10  day        750000 non-null  int64 
 11  month      750000 non-null  object
 12  duration   750000 non-null  int64 
 13  campaign   750000 non-null  int64 
 14  pdays      750000 non-null  int64 
 15  previous   750000 non-null  int64 
 16  poutcome   750000 non-null  object
 17  y          750000 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 103.0+ MB


Test Data Info:

In [9]:
# Get the counts of each class in the target variable
print("Target variable distribution (Counts):")
print(train_df['y'].value_counts())

print("\nTarget variable distribution (Percentages):")
print(train_df['y'].value_counts(normalize=True) * 100)

Target variable distribution (Counts):
y
0    659512
1     90488
Name: count, dtype: int64

Target variable distribution (Percentages):
y
0    87.934933
1    12.065067
Name: proportion, dtype: float64


In [10]:
pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 660.6 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.5 MB 393.8 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.5 MB 939.4 kB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.5 MB 1.1 MB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.5 MB 1.1 MB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.5 MB 1.3 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.5 MB 1.4 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.5 MB 1.6 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 1.7 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 1.7 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 1.7 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 1.7 MB/s eta 0:00:01
   ----------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier

# Load the data again (if not already in memory)
train_df = pd.read_csv(r'D:\manga\MACHINE,DEEP LEARNING\binary classification project\train.csv')
test_df = pd.read_csv(r'D:\manga\MACHINE,DEEP LEARNING\binary classification project\test.csv')

# Separate features (X) from the target (y)
X = train_df.drop('y', axis=1)
y = train_df['y']

# Separate numerical and categorical columns automatically
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Create the preprocessor using ColumnTransformer
# It applies OneHotEncoder to categorical features and leaves numerical ones alone
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Apply the preprocessing to both training and test data
# We fit the preprocessor on the training data only
X_encoded = preprocessor.fit_transform(X)

# We transform the test data using the fitted preprocessor from the training data
X_test_encoded = preprocessor.transform(test_df)

print("Shape of preprocessed training data:", X_encoded.shape)
print("Shape of preprocessed test data:", X_test_encoded.shape)

Shape of preprocessed training data: (750000, 52)
Shape of preprocessed test data: (250000, 52)


In [13]:
# Split the preprocessed training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

# Instantiate the LightGBM classifier with class_weight='balanced'
model = LGBMClassifier(class_weight='balanced', random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

print("\nModel training completed successfully! 🎉")

[LightGBM] [Info] Number of positive: 72390, number of negative: 527610
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.131869 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1296
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 52
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000

Model training completed successfully! 🎉


In [14]:
from sklearn.metrics import roc_auc_score

# Predict probabilities on the validation set
# We need the probability of the positive class (1), which is the second column [:, 1]
val_probabilities = model.predict_proba(X_val)[:, 1]

# Calculate the ROC AUC score
auc_score = roc_auc_score(y_val, val_probabilities)

print(f"The ROC AUC Score on the validation set is: {auc_score:.5f}")

The ROC AUC Score on the validation set is: 0.96484


In [17]:
# Define the full path for the submission file using a raw string
output_path = r'D:\manga\MACHINE,DEEP LEARNING\binary classification project\submission.csv'

# Predict probabilities on the preprocessed test data
test_probabilities = model.predict_proba(X_test_encoded)[:, 1]

# Create a DataFrame for the submission file
submission_df = pd.DataFrame({'id': test_df['id'], 'y': test_probabilities})

# Save the DataFrame to the specified path
submission_df.to_csv(output_path, index=False)

print(f"Submission file saved successfully to: {output_path} ✅")

Submission file saved successfully to: D:\manga\MACHINE,DEEP LEARNING\binary classification project\submission.csv ✅
